In [1]:
from azureml.core import Workspace, Experiment

In [2]:
ws = Workspace.get(name="quick-starts-ws-119614")
exp = Experiment(workspace=ws, name="hyperdrive")


WARNING - Port '8400' is taken with error '[Errno 48] Address already in use'. Trying with the next one
WARNING - Port '8401' is taken with error '[Errno 48] Address already in use'. Trying with the next one
WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


Performing interactive authentication. Please follow the instructions on the terminal.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [13]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

cluster_name = 'hd-cluster'

try:
    compute_target = ComputeTarget(ws, cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

In [14]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

In [15]:
ps = RandomParameterSampling(parameter_space = {'--C': uniform(0.1,0.9), '--max_iter': choice(25, 50, 100, 150)})

policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

est = SKLearn(source_directory = './', entry_script = 'train.py', compute_target = compute_target)

hyperdrive_config = HyperDriveConfig(hyperparameter_sampling = ps, primary_metric_name='Accuracy', primary_metric_goal= PrimaryMetricGoal.MAXIMIZE, policy = policy, max_total_runs = 50, max_concurrent_runs = 8, estimator = est  )



In [16]:
experiment_1 = exp.submit(config = hyperdrive_config)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [17]:
RunDetails(experiment_1).show()
#experiment_1.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
